In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
import numpy as np
import pandas as pd
import mplfinance as mpl
import seaborn as sns
from scipy import stats
from pathlib import Path
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")

## Data cleaning and wrangling: 

In [ ]:
#es0 = pd.read_csv('/Users/admin/Desktop/Weiterbildung/backtest/Data/e-mini/glbx-mdp3-20100606-20170430.ohlcv-1m.csv') #2010 
#es1 = pd.read_csv('/Users/admin/Desktop/Weiterbildung/backtest/Data/e-mini/e_mini_1m.csv') # 2017 
#es2 = pd.read_csv('/Users/admin/Desktop/Weiterbildung/backtest/Data/e-mini/glbx-mdp3-20241001-20250302.ohlcv-1m.csv') # #2024

In [ ]:
# es_combined = pd.concat([es0, es1, es2], ignore_index=True)
# es_combined.to_csv('/Users/admin/Desktop/Weiterbildung/backtest/Data/es_e-mini_comined.csv', index=False)

In [ ]:
es_path = "src/datasets/es_e-mini_comined.csv"
nq_path = "src/datasets/glbx-mdp3-20100606-20250302.ohlcv-1m.csv"

In [ ]:
def load_and_convert(file_path, tz='US/Eastern'):
    """
    Load a market data CSV file, convert timestamps to a specific timezone,
    and filter rows where the symbol has exactly 4 characters.

    Parameters:
    - file_path (str or Path): Path to the CSV file containing market data.
    - tz (str): Timezone to convert the timestamp column to. Default is 'US/Eastern'.

    Returns:
    - pd.DataFrame: Processed DataFrame with timezone-aware datetime values.
    """
    file_path = Path(file_path)
    df = pd.read_csv(file_path, parse_dates=['ts_event'])

    # Localize if naive, otherwise just convert
    if df['ts_event'].dt.tz is None:
        df['ts_event'] = df['ts_event'].dt.tz_localize('UTC').dt.tz_convert(tz)
    else:
        df['ts_event'] = df['ts_event'].dt.tz_convert(tz)

    # Filter rows where 'symbol' has exactly 4 characters
    df = df[df['symbol'].apply(lambda x: len(str(x)) == 4)]

    return df


In [ ]:
es_df = load_and_convert(es_path)
nq_df = load_and_convert(nq_path)

In [ ]:
#mpl.plot(sunday, volume=True ,type='candle', tight_layout=True, style='charles', figsize=(18,7), warn_too_much_data=len(sunday)+1)

March (H), June (M), September (U), December (Z)


In [ ]:
def filter_nq_contracts(dataframe):
    """
    filtering dataframe for contracts NQH, NQM, NQU, NQZ,
    """

    dataframe = dataframe.copy()
    dataframe['ts_event'] = pd.to_datetime(dataframe['ts_event'])

    dataframe['month'] = dataframe['ts_event'].dt.month
    dataframe['day'] = dataframe['ts_event'].dt.day

    mask_h = (((dataframe['month'] == 12) & (dataframe['day'] >= 15)) | ((dataframe['month'] == 1) | (dataframe['month'] == 2) | ((dataframe['month'] == 3) & (dataframe['day'] <= 21)))) & dataframe['symbol'].str.startswith('NQH')
    mask_m = (((dataframe['month'] == 3) & (dataframe['day'] >= 16)) | (dataframe['month'] == 4) | (dataframe['month'] == 5) | ((dataframe['month'] == 6) & (dataframe['day'] <= 20))) & dataframe['symbol'].str.startswith('NQM')
    mask_u = (((dataframe['month'] == 6) & (dataframe['day'] >= 16)) | (dataframe['month'] == 7) | (dataframe['month'] == 8) | ((dataframe['month'] == 9) & (dataframe['day'] <= 19))) & dataframe['symbol'].str.startswith('NQU')
    mask_z = (((dataframe['month'] == 9) & (dataframe['day'] >= 15)) | (dataframe['month'] == 10) | (dataframe['month'] == 11) | ((dataframe['month'] == 12) & (dataframe['day'] <= 19))) & dataframe['symbol'].str.startswith('NQZ')

    valid_mask = mask_h | mask_m | mask_u | mask_z

    dataframe.loc[mask_h, 'sym'] = 'NQH'
    dataframe.loc[mask_m, 'sym'] = 'NQM'
    dataframe.loc[mask_u, 'sym'] = 'NQU'
    dataframe.loc[mask_z, 'sym'] = 'NQZ'

    return dataframe[valid_mask].drop(columns=['month', 'day'])


In [ ]:
def filter_es_contracts(dataframe):
    """
    filtering dataframe for contracts ESH, ESM, ESU, ESZ,
    """

    dataframe = dataframe.copy()
    dataframe['ts_event'] = pd.to_datetime(dataframe['ts_event'])

    dataframe['month'] = dataframe['ts_event'].dt.month
    dataframe['day'] = dataframe['ts_event'].dt.day

    mask_h = (((dataframe['month'] == 12) & (dataframe['day'] >= 15)) | ((dataframe['month'] == 1) | (dataframe['month'] == 2) | ((dataframe['month'] == 3) & (dataframe['day'] <= 21)))) & dataframe['symbol'].str.startswith('ESH')
    mask_m = (((dataframe['month'] == 3) & (dataframe['day'] >= 16)) | (dataframe['month'] == 4) | (dataframe['month'] == 5) | ((dataframe['month'] == 6) & (dataframe['day'] <= 20))) & dataframe['symbol'].str.startswith('ESM')
    mask_u = (((dataframe['month'] == 6) & (dataframe['day'] >= 16)) | (dataframe['month'] == 7) | (dataframe['month'] == 8) | ((dataframe['month'] == 9) & (dataframe['day'] <= 19))) & dataframe['symbol'].str.startswith('ESU')
    mask_z = (((dataframe['month'] == 9) & (dataframe['day'] >= 15)) | (dataframe['month'] == 10) | (dataframe['month'] == 11) | ((dataframe['month'] == 12) & (dataframe['day'] <= 19))) & dataframe['symbol'].str.startswith('ESZ')

    valid_mask = mask_h | mask_m | mask_u | mask_z

    dataframe.loc[mask_h, 'sym'] = 'ESH'
    dataframe.loc[mask_m, 'sym'] = 'ESM'
    dataframe.loc[mask_u, 'sym'] = 'ESU'
    dataframe.loc[mask_z, 'sym'] = 'ESZ'

    return dataframe[valid_mask].drop(columns=['month', 'day'])

In [ ]:
filtered_nq = filter_nq_contracts(nq_df)
filtered_es = filter_es_contracts(es_df)

In [ ]:
filtered_es.shape

In [ ]:
#mpl.plot(sunday,volume=True ,type='candle', tight_layout=True,style='charles',  figsize=(18,7), warn_too_much_data=len(sunday)+1) 

## Date filter: 

In [ ]:
def set_dates(dataframe, start_date="2020-06-06", end_date="2025-03-02"):
    return dataframe[(dataframe['ts_event'] >= pd.to_datetime(start_date).tz_localize("US/Eastern")) & (dataframe['ts_event'] <= pd.to_datetime(end_date).tz_localize("US/Eastern"))]

In [ ]:
limited_nq = set_dates(filtered_nq).set_index("ts_event")
limited_es = set_dates(filtered_es).set_index("ts_event")

## Initialize RTH session dataframe: 

In [ ]:
rth_session_nq = limited_nq.between_time("09:30", "16:00")
rth_session_es = limited_es.between_time("09:30", "16:00")

In [ ]:
# mpl.plot(rth_session,volume=True,type='candle', tight_layout=True, style='charles', figsize=(18,7), warn_too_much_data=len(rth_session)+1)

## Defining overnight: 


In [ ]:
def calculate_ovn_high_low(dataframe):
    """
    Calculate overnight high and low values for each session in the DataFrame.

    Parameters:
    df (pd.DataFrame): A DataFrame with a DatetimeIndex and at least 'high' and 'low' columns.

    Returns:
    pd.DataFrame: A DataFrame containing 'ts_event', 'ovn_high', and 'ovn_low' for each overnight session.
    """
    overnight_results = []
    groups = list(dataframe.groupby(dataframe.index.date))

    for i in range(0, len(groups) - 1):
        (_, group1), (date2, group2) = groups[i], groups[i + 1]

        curr_day_data = group1.between_time("16:00", "23:59")
        next_day_data = group2.between_time("00:00", "09:29")

        maximum = max(curr_day_data['high'].max(), next_day_data['high'].max())
        minimum = min(curr_day_data['low'].min(), next_day_data['low'].min())

        overnight_results.append({
            'ts_event': pd.to_datetime(date2),
            'ovn_high': maximum,
            'ovn_low': minimum,
        })

    return pd.DataFrame(overnight_results)


In [ ]:
overnight_df_nq = calculate_ovn_high_low(rth_session_nq).set_index('ts_event')
overnight_df_es = calculate_ovn_high_low(rth_session_es).set_index('ts_event')

In [ ]:
rth_session_nq

## Merging data: 


In [ ]:
def merge_df(rth_df: pd.DataFrame, overnight_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merges RTH data with overnight data based on the date.

    Parameters:
        rth_df (pd.DataFrame): DataFrame with a 'ts_event' index (datetime with time).
        overnight_df (pd.DataFrame): DataFrame with a 'ts_event' index (date or datetime).

    Returns:
        pd.DataFrame: The merged DataFrame with cleaned columns.
    """

    rth_df = rth_df.reset_index().copy()
    rth_df['date_only'] = pd.to_datetime(rth_df['ts_event']).dt.date

    overnight_df = overnight_df.reset_index().copy()
    overnight_df['ts_event'] = pd.to_datetime(overnight_df['ts_event']).dt.date

    merged = pd.merge(
        rth_df,
        overnight_df,
        how='left',
        left_on='date_only',
        right_on='ts_event',
        suffixes=('', '_overnight')
    )

    merged['ts_event_overnight'] = pd.to_datetime(merged['ts_event_overnight'])

    cols_to_drop = [
        col for col in ['date_only', 'rtype', 'publisher_id']
        if col in merged.columns
    ]

    merged.drop(columns=cols_to_drop, inplace=True)

    return merged


In [ ]:
nq_merged = merge_df(rth_session_nq, overnight_df_nq)
#es_merged = merge_df(rth_session_es, overnight_df_es)
#nq_merged[nq_merged['ovn_low'].isna()]

## Calculate broke of OVN High and OVN Low: 

In [ ]:
def calc_ovn_broke(dataframe):
    """
    Calculates the overnight breakout condition for each group in the DataFrame and annotates the result.

    Parameters:
    - dataframe (pd.DataFrame): Input DataFrame with intraday data and overnight high/low levels.
    - result_col (str): Column name where results will be stored (created if not present).

    Returns:
    - pd.DataFrame: Modified DataFrame with the new result column filled.
    """
    dataframe['OVN break'] = 0  

    for date, group in dataframe.groupby('ts_event'):
        high_break_index = None
        low_break_index = None

        for i, row in group.iterrows():
            if row['close'] >= row['ovn_high'] and high_break_index is None:
                high_break_index = i
            if row['close'] <= row['ovn_low'] and low_break_index is None:
                low_break_index = i
            if high_break_index is not None and low_break_index is not None:
                break

        if high_break_index is not None and low_break_index is not None:
            dataframe.at[max(high_break_index, low_break_index), 'OVN break'] = 3
        elif high_break_index is not None:
            dataframe.at[high_break_index, 'OVN break'] = 2
        elif low_break_index is not None:
            dataframe.at[low_break_index, 'OVN break'] = 1

    return dataframe


## Calculate output: 

In [ ]:
ovn_broke_nq = calc_ovn_broke(nq_merged).dropna()
#ovn_broke_es = calc_ovn_broke(es_merged)

In [ ]:
ovn_broke_nq

In [ ]:
def get_distribution_info(df: pd.DataFrame) -> dict:
    """
    Give information about the distribution of 'OVN break' events in a DataFrame.

    Parameters:
    ----------
    df : pd.DataFrame
        A DataFrame containing at least the columns 'ts_event_y' (datetime) and 'OVN break' (int).

    Returns:
    -------
    dict
        A dictionary with:
        - The min and max observation dates,
        - Total number of observations (excluding Sundays),
        - Number of broken days and not broken days,
        - Count of OVN High, Low, and both High & Low breaks.
    """
    # Ensure required columns are present
    required_columns = {'ts_event_overnight', 'OVN break'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

    # Filter out Sundays
    weekday_df = df[df['ts_event_overnight'].dt.weekday != 6]
    
    # Unique number of observation days excluding Sundays
    num_observations = weekday_df['ts_event_overnight'].dt.date.nunique()

    # Filter only broken days (OVN break != 0)
    broken_df = df[df['OVN break'] != 0]
    broken_counts = broken_df['OVN break'].value_counts()

    # Extract specific break types with defaults
    ovn_high_broken = broken_counts.get(2, 0)
    ovn_low_broken = broken_counts.get(1, 0)
    ovn_high_low_broken = broken_counts.get(3, 0)

    num_broken_days = broken_counts.sum()
    num_not_broken = num_observations - num_broken_days

    return {
        'Date Range': f"{df['ts_event_overnight'].dt.date.min()} - {df['ts_event_overnight'].dt.date.max()}",
        'Observation Days': num_observations,
        'Broken Days': num_broken_days,
        'Not Broken Days': num_not_broken,
        'OVN High Broken': ovn_high_broken,
        'OVN Low Broken': ovn_low_broken,
        'OVN High & Low Broken': ovn_high_low_broken
    }

In [ ]:
get_dist_info = get_distribution_info(ovn_broke_nq)

## Plotting pie chart for each asset: 


In [ ]:
def plot_pie(date_range, ovn_high_broken, ovn_low_broken, ovn_high_low_broken, num_not_broken):
    """
    Plot a pie chart visualizing the distribution of overnight breakout scenarios.

    Parameters:
    ----------
    count_series : pd.Series
        A Series containing counts of 'OVN break' values (1=low, 2=high, 3=both).
    not_broken_days : int
        The number of days when no overnight breakout occurred.

    Returns:
    -------
    None
    """
    # Define custom colors for each segment
    segment_colors = ['#267a17', '#0d47a1', '#D22B2B', '#c9c9c9'] 

    # Labels with dynamic values (use .get with default to avoid KeyError)
    labels = [
        f"High broken:\n{ovn_high_broken} days",
        f"Low broken:\n{ovn_low_broken} days",
        f"High & Low broken:\n{ovn_high_low_broken} days",
        f"None broken:\n{num_not_broken} days"
    ]

    # Values to plot (in matching order with labels/colors)
    pie_values = [
        ovn_high_broken,
        ovn_low_broken,
        ovn_high_low_broken,
        num_not_broken
    ]

    # Pie chart configuration
    plt.figure(figsize=(7, 7))
    plt.pie(
        pie_values,
        autopct='%1.1f%%',
        startangle=117,
        explode=[0.03] * 4,
        colors=segment_colors,
        labels=labels
    )
    plt.title(f"NQ Overnight Breakout Probability\n{date_range}", fontsize=15)
    plt.tight_layout()
    plt.show()


In [ ]:
result = plot_pie(date_range=get_dist_info['Date Range'],
                  ovn_high_broken=get_dist_info['OVN High Broken'], 
                  ovn_low_broken=get_dist_info['OVN Low Broken'], 
                  ovn_high_low_broken= get_dist_info['OVN High & Low Broken'], 
                  num_not_broken=get_dist_info['Observation Days']
                  )

result

## Data wrangling for building a chart with a time-based x-axis: 


In [ ]:
dist_df = ovn_broke_nq[['ts_event', 'OVN break']]

dist_df['Time'] = pd.to_datetime(dist_df['ts_event'], format='%H:%M:%S').dt.time

both_breakout_df = dist_df[dist_df['OVN break'] == 3]
both_breakout_df['Minutes'] = both_breakout_df['Time'].apply(lambda t: t.hour * 60 + t.minute)

high_breakout_df = dist_df[dist_df['OVN break'] == 2]
high_breakout_df['Minutes'] = high_breakout_df['Time'].apply(lambda t: t.hour * 60 + t.minute)

low_breakout_df = dist_df[dist_df['OVN break'] == 1]
low_breakout_df['Minutes'] = low_breakout_df['Time'].apply(lambda t: t.hour * 60 + t.minute)


grouped = dist_df.groupby(['Time', 'OVN break'])['OVN break'].count().reset_index(name='Count')
grouped = grouped.sort_values('Time')
grouped['Minutes'] = grouped['Time'].apply(lambda t: t.hour * 60 + t.minute)

In [ ]:
grouped

## Plot graph for each asset: 

In [ ]:
plt.figure(figsize=(11, 4))
 
group_1 = grouped[grouped['OVN break'] == 1.0]
#plt.bar(group_1['Minutes'], group_1['Count'], width=2, alpha=.65, label='OVN breakout low', color='blue')
group_2 = grouped[grouped['OVN break'] == 2.0]
plt.bar(group_2['Minutes'], group_2['Count'], width=2, alpha=.7, label='OVN breakout high', color='green')
group_3 = grouped[grouped['OVN break'] == 3.0]
#plt.bar(group_3['Minutes'], group_3['Count'], width=2, alpha=0.5, label='OVN breakout both, second breakout', color='#D22B2B')

start_min = 9 * 60 + 30   
end_min = 16 * 60         
xticks = list(range(start_min, end_min + 1, 5)) 
xticklabels = [f'{h//60:02d}:{h%60:02d}' for h in xticks]

#plt.axvspan(xmin=570, xmax=629, color='red', alpha=0.22, label='High breakout 09:30–10:28') # NQ OVN breakout high
#plt.axvspan(xmin=570, xmax=596, color='red', alpha=0.2, label='First breakout 09:30–09:56') # NQ OVN breakout low 

plt.axvspan(xmin=570, xmax=596, color='red', alpha=0.22,label='First breakout 09:30–10:53') # OVN breakout high
#plt.axvspan(xmin=570, xmax=660, color='red', alpha=0.22, label='Low breakout 09:30–11:00') # OVN breakout low 

plt.xticks(xticks, xticklabels, rotation=45, fontsize=6)
plt.xlim(start_min - 1.5, end_min + 1.6)
plt.yticks(ticks=range(0, 10, 5), fontsize=6)
plt.xlabel('RTH session time lapse')
plt.ylabel('Amount of occurencies')
plt.title('NQ Overnight breakout distribution')
plt.legend()
plt.style.use('seaborn-v0_8')
plt.grid(visible=True, which='both')
plt.show()

## Boxplot for each asset nq and es: 

In [ ]:
plt.style.use('seaborn-v0_8')
fig, ax = plt.subplots(figsize=(11, 4), dpi=500)

data = [low_breakout_df['Minutes'], high_breakout_df['Minutes']]
labels = ['Low breakout', 'High breakout']
colors = ['blue', 'green']

box = ax.boxplot(
    data,
    vert=False,
    patch_artist=True,
    labels=labels,
    widths=0.6,
    medianprops=dict(color='red', linewidth=1.5)
)

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.58)

start_min = 9 * 60 + 30   # 570
end_min = 16 * 60         # 960
xticks = list(range(start_min, end_min + 1, 15))
xticklabels = [f'{h//60:02d}:{h%60:02d}' for h in xticks]
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, rotation=45, fontsize=6)

ax.set_xlim(start_min - 1.5, end_min + 1.6)
ax.set_yticks([1, 2])
ax.set_yticklabels(labels, fontsize=6)

ax.set_xlabel('RTH session time lapse')
ax.set_ylabel('Amount of occurencies')
ax.set_title('Boxplot of NQ Overnight breakout distribution')

ax.grid(visible=True, which='both')

legend_elements = [
    Patch(color='red', label='Median')
]
ax.legend(handles=legend_elements, fontsize=6)

#plt.tight_layout()
plt.show()


## Calculate quantile 75%: 

In [ ]:
q3high = both_breakout_df['Minutes'].quantile(0.75)
q3low = both_breakout_df['Minutes'].quantile(0.75)
print(f"high {q3high / 60:.2f} hours")
print(f"low {q3low / 60:.2f} hours")
print(q3high)
print(q3low)

#low_breakout_df['Minutes']

In [ ]:

plt.style.use('seaborn-v0_8')

fig, ax = plt.subplots(figsize=(11, 2.8), dpi=500)

data = [both_breakout_df['Minutes']]
labels = ['Both breakout']
colors = ['red']

box = ax.boxplot(
    data,
    vert=False,
    patch_artist=True,
    labels=labels,
    widths=0.6,
    medianprops=dict(color='red', linewidth=1.5)
)


for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.58)

start_min = 9 * 60 + 30   # 570
end_min = 16 * 60         # 960
xticks = list(range(start_min, end_min + 1, 15))
xticklabels = [f'{h//60:02d}:{h%60:02d}' for h in xticks]
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, rotation=45, fontsize=6)

ax.set_xlim(start_min - 1.5, end_min + 1.6)
ax.set_yticks([1])  
ax.set_yticklabels(labels, fontsize=6)

ax.set_xlabel('RTH session time lapse')
ax.set_ylabel('Amount of occurencies')
ax.set_title('Boxplot of NQ Both-side Overnight breakout distribution. First breakout')

ax.grid(visible=True, which='both')
legend_elements = [Patch(color='red', label='Median')]
ax.legend(handles=legend_elements, fontsize=6)

plt.tight_layout()
plt.show()